## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from pathlib import Path

In [3]:
if os.name == 'nt':
    PATH = str(Path.home()) + "\\Development\\kaggle\\dogscats\\data\\"
else:
    PATH = "data/dogscats/"
    
sz=224
arch=vgg16
bs=64

In [4]:
# Uncomment the below if you need to reset your precomputed activations
if os.name == 'nt':
    ## warning: will remove everything under the following path (windows):
    #delPath = f"{PATH}tmp\\"
    #!rmdir /S /Q "{delPath}"
    ;
else:
    #!rm -rf {PATH}tmp
    ;

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [6]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [7]:
%time learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       0.04957  0.03127  0.98853]                                                                                  
[ 1.       0.05194  0.02928  0.99011]                                                                                  
[ 2.       0.04766  0.02834  0.99011]                                                                                  

Wall time: 14.5 s


In [8]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [9]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [10]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       0.05197  0.03004  0.98952]                                                                                  
[ 1.       0.03972  0.02785  0.99051]                                                                                  



In [11]:
learn.precompute=False

In [12]:
%time learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.05336  0.03081  0.98833]                                                                                  

Wall time: 2min 2s


In [13]:
learn.unfreeze()

In [14]:
lr=np.array([1e-4,1e-3,1e-2])

In [15]:
%time learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.07256  0.0286   0.98912]                                                                                  

Wall time: 4min 44s


In [16]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.05463  0.02681  0.9909 ]                                                                                  
[ 1.       0.03719  0.0241   0.99031]                                                                                  
[ 2.       0.03202  0.02369  0.9911 ]                                                                                  
[ 3.       0.07117  0.02401  0.99209]                                                                                  
[ 4.       0.02895  0.02278  0.99208]                                                                                  
[ 5.       0.02487  0.02309  0.99247]                                                                                  
[ 6.       0.02252  0.0225   0.99247]                                                                                  

Wall time: 33min 7s


In [17]:
%time learn.fit(lr, 3, cycle_len=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[ 0.       0.03181  0.02285  0.99208]                                                                                  
[ 1.       0.02869  0.02291  0.99168]                                                                                  
[ 2.       0.02297  0.02243  0.99168]                                                                                  
[ 3.       0.02365  0.02598  0.99128]                                                                                  
[ 4.       0.0218   0.02505  0.99208]                                                                                  
[ 5.       0.01362  0.0278   0.99049]                                                                                  
[ 6.       0.01833  0.02471  0.99267]                                                                                  
[ 7.       0.07713  0.02935  0.99069]                                                                                  
[ 8.       0.01583  0.02594  0.99267]   

In [19]:
%time log_preds_augmented,y = learn.TTA()
log_preds = log_preds_augmented.mean(axis=0)
accuracy(log_preds, y)

Wall time: 1min 23s                                                                                                    


0.99283296834561019